## Autor: Pablo Veloz M.

In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from scipy import stats 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.patches as mpatches
#import lec4_graphs as gfx
import ipywidgets as widgets 
from ipywidgets import interact, interact_manual
from IPython.display import display
from ipywidgets import Checkbox
#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
df = pd.read_csv('southafricanheart.csv', encoding = 'ISO-8859-1',sep=',', engine='python')

In [4]:
df=df.drop("Unnamed: 0",axis=1)

In [5]:
df.head(5)


,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


# Desafío 1: Preparar el ambiente de trabajo
* Se detallan los pasos a seguir
* tip: Los tips o sugerencias preceden de tip
* Se generan dos notebooks, uno con las soluciones y otro con los ejercicios.

# Desafío 2
* A continuación se presenta el siguiente modelo a estimar:
$log\left(\frac{Pr(chd=1)}{1-Pr(chd=1)}\right)=\beta_{0}+\beta_{1}\cdot famhist$

Para ello ejecute los siguientes pasos:
1. Recodifique famhist a dummy, asignando 1 a la categoría minoritaria.
2. Utilice smf.logit para estimar el modelo.
3. Implemente una función inverse_logit que realize el mapeo de log-odds a probabilidad.
4. Con el modelo estimado, responda lo siguiente:

    * ¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria?
    * ¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?
    * ¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?
    * Replique el modelo con smf.ols y comente las similitudes entre los coeficientes estimados.
    tip: utilice $\beta_{0}=4$
    * Estime el mismo modelo con LPM

### - Recodifique famhist a dummy, asignando 1 a la categoría minoritaria.

In [6]:
df["famhist"].value_counts()

Absent     270
Present    192
Name: famhist, dtype: int64

In [7]:
df["dummy_famhist"]= np.where(df["famhist"]=="Present",1,0)
df.head(5)

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd,dummy_famhist
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1,0
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0,1
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1,1


### - Utilice smf.logit para estimar el modelo.

In [8]:
def concise_summary(mod, print_fit=True):
    #guardamos los parámetros asociados a estadísticas de ajuste
    fit = pd.DataFrame({'Statistics': mod.summary2().tables[0][2][2:],
    'Value': mod.summary2().tables[0][3][2:]})
    # guardamos los parámetros estimados por cada regresor.
    estimates = pd.DataFrame(mod.summary2().tables[1].loc[:, 'Coef.': 'Std.Err.'])
    # imprimir fit es opcional
    if print_fit is True:
        print("\nGoodness of Fit statistics\n", fit)
        print("\nPoint Estimates\n\n", estimates)
    # solicitemos las características del modelo


In [9]:
m1_logit = smf.logit('chd ~ dummy_famhist', df).fit()
concise_summary(m1_logit)

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5

Goodness of Fit statistics
         Statistics       Value
2             BIC:    574.1655
3  Log-Likelihood:     -280.95
4         LL-Null:     -298.05
5     LLR p-value:  4.9371e-09
6           Scale:      1.0000
7                             

Point Estimates

                   Coef.  Std.Err.
Intercept     -1.168993  0.143106
dummy_famhist  1.168993  0.203255


#### Implemente una función inverse_logit que realize el mapeo de log-odds a probabilidad.
1. Calcular el log odds promedio cuando dummy_famhist es igual al promedio

In [10]:
dummy_famhist_mean = df['dummy_famhist'].mean()
print("La media es de ", round(dummy_famhist_mean, 2))

La media es de  0.42


In [11]:
estimate_chd = m1_logit.params['Intercept'] + (m1_logit.params['dummy_famhist'] * dummy_famhist_mean)
print("El log odds logit para el promedio estimado es de:", round(estimate_chd, 2))

El log odds logit para el promedio estimado es de: -0.68


In [12]:
estimate_chd_1 = m1_logit.params['Intercept'] + (m1_logit.params['dummy_famhist'] * 1)
print("El log odds logit estimado es de ", round(estimate_chd_1, 4))

El log odds logit estimado es de  0.0


In [13]:
m1_logit.params['dummy_famhist'] * 1

1.1689930854299098

2. Convertir nuestro log odds estimado en una probabilidad

#### Función inverse_logit.

In [14]:
def invlogit(x):
    return 1 / (1+np.exp(-x))


¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria?

In [15]:
estimate_con_antecedentes_chd_1 = m1_logit.params['Intercept'] + (m1_logit.params['dummy_famhist'] * 1)
print("El log odds logit estimado es de ", round(estimate_con_antecedentes_chd_1, 4))
print("La probabilidad promedio de presentar una enfermedad sin antecedentes es de:",round(invlogit(estimate_con_antecedentes_chd_1), 2))
p_con_ant_enf_cor=round(invlogit(estimate_con_antecedentes_chd_1), 4)

El log odds logit estimado es de  0.0
La probabilidad promedio de presentar una enfermedad sin antecedentes es de: 0.5


¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria

In [16]:
estimate_sin_antecedentes_chd_1 = m1_logit.params['Intercept'] + (m1_logit.params['dummy_famhist'] * 0)
print("El log odds logit estimado es de ", round(estimate_sin_antecedentes_chd_1, 4))
print("La probabilidad promedio de presentar una enfermedad sin antecedentes es de:",round(invlogit(estimate_sin_antecedentes_chd_1), 2))
p_sin_ant_enf_cor=round(invlogit(estimate_sin_antecedentes_chd_1), 2)

El log odds logit estimado es de  -1.169
La probabilidad promedio de presentar una enfermedad sin antecedentes es de: 0.24


In [17]:
#print("La probabilidad promedio de presentar una enfermedad sin antecedentes es de:",round(1-invlogit(estimate_chd), 2))
#p_sin_ant_enf_cor=round(1-invlogit(estimate_chd), 2)

#### ¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?



In [18]:
print("La diferencia en la probabilidad es:",abs(p_con_ant_enf_cor-p_sin_ant_enf_cor))
p_logit_diff=abs(p_con_ant_enf_cor-p_sin_ant_enf_cor)

La diferencia en la probabilidad es: 0.26


Replique el modelo con smf.ols y comente las similitudes entre los coeficientes estimados. tip: utilice  $\beta/4$ 
Estime el mismo modelo con LPM

In [19]:
def concise_summary_ols(mod, print_fit=True):
    #guardamos los parámetros asociados a estadísticas de ajuste
    fit = pd.DataFrame({'Statistics': mod.summary2().tables[0][2][2:],
    'Value': mod.summary2().tables[0][3][2:]})
    # guardamos los parámetros estimados por cada regresor.
    estimates = pd.DataFrame(mod.summary2().tables[1].loc[:, 'Coef.': 'Std.Err.'])
    # imprimir fit es opcional
    if print_fit is True:
        print("\nGoodness of Fit statistics\n", fit)
        print("\nPoint Estimates\n\n", estimates)
    # solicitemos las características del modelo
    

In [20]:
m1_ols = smf.ols('chd ~ dummy_famhist', df).fit()
concise_summary_ols(m1_ols)


Goodness of Fit statistics
             Statistics     Value
2                 BIC:  601.4437
3      Log-Likelihood:   -294.59
4         F-statistic:     36.86
5  Prob (F-statistic):  2.66e-09
6               Scale:   0.21050

Point Estimates

                   Coef.  Std.Err.
Intercept      0.237037  0.027922
dummy_famhist  0.262963  0.043313


In [34]:
estimate_chd_ols = m1_ols.params['Intercept'] + (m1_ols.params['dummy_famhist'] * 1)
print("El log odds estimado ols es de ", round(estimate_chd_ols, 2))



El log odds estimado ols es de  0.5


In [35]:
print("La probabilidad en ols promedio de presentar una enfermedad con antecedentes es de:",round(invlogit(estimate_chd_ols), 2))
p_ols_con_ant_enf_cor=round(invlogit(estimate_chd_ols), 2)

La probabilidad en ols promedio de presentar una enfermedad con antecedentes es de: 0.62


In [43]:
estimate_sin_chd_ols = m1_ols.params['Intercept'] + (m1_ols.params['dummy_famhist'] * 0)

print("La probabilidad en ols promedio de presentar una enfermedad sin antecedentes es de:",round(invlogit(estimate_sin_chd_ols), 2))
p_ols_sin_ant_enf_cor=round(invlogit(estimate_sin_chd_ols), 2)

La probabilidad en ols promedio de presentar una enfermedad sin antecedentes es de: 0.56


In [44]:
print("la diferencia de las probabilidades en ols es de: ",p_ols_con_ant_enf_cor-p_ols_sin_ant_enf_cor)
p_ols_diff=p_ols_con_ant_enf_cor-p_ols_sin_ant_enf_cor

la diferencia de las probabilidades en ols es de:  0.05999999999999994


### Modelos logit vs Ols

In [45]:
lista_enfermedades=["Con antecedentes","Sin antecedentes","Diferencia"]
lista_logit=[p_con_ant_enf_cor,p_sin_ant_enf_cor,p_logit_diff]
lista_ols=[p_ols_con_ant_enf_cor,p_ols_sin_ant_enf_cor,p_ols_diff]
pd.DataFrame({'Chd':lista_enfermedades,'Modelo Logit': lista_logit,'Modelo Ols': lista_ols})

,Chd,Modelo Logit,Modelo Ols
0,Con antecedentes,0.50,0.62
1,Sin antecedentes,0.24,0.56
2,Diferencia,0.26,0.06


# Desafío 3: Estimación completa
* Implemente un modelo con la siguiente forma:
$log\left(\frac{Pr(chd=1)}{1-Pr(chd=1)}\right)=\beta_{0}+\sum_{j=1}^{N}\beta_{j}\cdot X$


* Depure el modelo manteniendo las variables con significancia estadística al 95%.
* Compare los estadísticos de bondad de ajuste entre ambos.
* Reporte de forma sucinta el efecto de las variables en el log-odds de tener una enfermedad coronaria.

In [26]:
modelo1_logit = smf.logit('chd ~ sbp+tobacco+ldl+adiposity+typea+obesity+alcohol+age+dummy_famhist', df).fit()

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


In [27]:
modelo1_logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      452
Method:                           MLE   Df Model:                            9
Date:                Tue, 30 Jul 2019   Pseudo R-squ.:                  0.2080
Time:                        17:22:06   Log-Likelihood:                -236.07
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 2.055e-22
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -6.1507      1.308     -4.701      0.000      -8.715      -3.587
sbp               0.0065      0.006      1.135      0.256      -0.005       0.018
tobacco           0.0794      0.027      2.984      0.003       0.027       0.132
ldl               0.1739      0.060      2.915      0.004       0.057       0.291
adiposity         0.0186      0.029      0.635      0.526      -0.039       0.076
typea             0.0396      0.012      3.214      0.001       0.015       0.064
obesity          -0.0629      0.044     -1.422      0.155      -0.150       0.024
alcohol           0.0001      0.004      0.027      0.978      -0.009       0.009
age               0.0452      0.012      3.728      0.000       0.021       0.069
dummy_famhist     0.9254      0.228      4.061      0.000       0.479       1.372
=================================================================================
"""

### Depurando variables con pvalues <0.05
   * tobacco
   * ldl
   * typea
   * age
   * dummy_famhist

In [28]:
modelo2_logit = smf.logit('chd ~ tobacco+ldl+typea+age+dummy_famhist', df).fit()

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6


In [29]:
modelo2_logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      456
Method:                           MLE   Df Model:                            5
Date:                Tue, 30 Jul 2019   Pseudo R-squ.:                  0.2020
Time:                        17:22:06   Log-Likelihood:                -237.84
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 2.554e-24
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -6.4464      0.921     -7.000      0.000      -8.251      -4.642
tobacco           0.0804      0.026      3.106      0.002       0.030       0.131
ldl               0.1620      0.055      2.947      0.003       0.054       0.270
typea             0.0371      0.012      3.051      0.002       0.013       0.061
age               0.0505      0.010      4.944      0.000       0.030       0.070
dummy_famhist     0.9082      0.226      4.023      0.000       0.466       1.351
=================================================================================
"""

#### Se observa que la variable dummy_famhist, tiene un mayor coeficientes ($\beta$) entre todas las variables estudiadas. La menor lo reporta typea.

### Compare los estadísticos de bondad de ajuste entre ambos.


In [30]:
lista_r2=[0.2080,0.2020]
lista_verosimilitud=[-236.07,-237.84]
modelos=["Modelo 1","Modelo Depurado"]
tmp2=pd.DataFrame({'Modelos':modelos,'r^2': lista_r2,'verosimilitud': lista_verosimilitud})
tmp2

,Modelos,r^2,verosimilitud
0,Modelo 1,0.208,-236.07
1,Modelo Depurado,0.202,-237.84


# Desafío 4: Estimación de perfiles
- A partir del modelo depurado, genere las estimaciones en log-odds y posteriormente transfórmelas a probabilidades con inverse_logit . Los perfiles a estimar son los siguientes:
    * La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra.
    * La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.
    * La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.

### La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra.

In [31]:
def modelo_carac_similares(columnas_seleccionadas,dataframe):
    global modelo2_logit
    intercepto=modelo2_logit.params['Intercept']
    estimate_chd=0
    inversa=0
    for i in columnas_seleccionadas:
        estimate_chd += modelo2_logit.params[i] * dataframe[i].mean()
    estimate2=estimate_chd+intercepto
    inversa= 1 / (1+np.exp(-estimate2))
    print("La probabilidad de tener una enfermedad coronaria con caracteristicas similares es: ",round(inversa,3))
modelo_carac_similares(["tobacco","ldl","typea","age","dummy_famhist"],df)


La probabilidad de tener una enfermedad coronaria con caracteristicas similares es:  0.294


### La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.

In [46]:
def modelo_altos_niveles_ldl(columnas_seleccionadas,dataframe):
    global modelo2_logit
    intercepto=modelo2_logit.params['Intercept']
    estimate_chd=0
    inversa=0
    for i in columnas_seleccionadas:
        if i=="ldl":
            estimate_chd += modelo2_logit.params[i] * max(df["ldl"])
        else:
            estimate_chd += modelo2_logit.params[i] * df[i].mean()
    estimate2=estimate_chd+intercepto
    inversa= 1 / (1+np.exp(-estimate2))
    print("La probabilidad de tener una enfermedad coronaria con altos niveles ldl es: ",round(inversa,3))
modelo_altos_niveles_ldl(["tobacco","ldl","typea","age","dummy_famhist"],df)

La probabilidad de tener una enfermedad coronaria con altos niveles ldl es:  0.698


### La probabilidad de tener una enfemerdad coronaria para un individuo con __bajos__ niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes

In [49]:
def modelo_bajos_niveles_ldl(columnas_seleccionadas,dataframe):
    global modelo2_logit
    intercepto=modelo2_logit.params['Intercept']
    estimate_chd=0
    inversa=0
    for i in columnas_seleccionadas:
        if i == "ldl":
            estimate_chd += modelo2_logit.params[i] * min(df["ldl"])
        else:
            estimate_chd += modelo2_logit.params[i] * df[i].mean()
        
    estimate2=estimate_chd+intercepto
    inversa= 1 / (1+np.exp(-estimate2))
    print("La probabilidad de tener una enfermedad coronaria con bajos niveles ldl es: ",round(inversa,3))
modelo_bajos_niveles_ldl(["tobacco","ldl","typea","age","dummy_famhist"],df)

La probabilidad de tener una enfermedad coronaria con bajos niveles ldl es:  0.184
